# Masking E-voting Database

## Implementation

### Database

In [25]:
# Database for votes
# Each row in votes database contains these information:
# 1. timestamp
# 2. voter's id
# 3. vote for candidate 1 (0 or 1)
# 4. vote for candidate 2 (0 or 1)
# .
# .
# .
# n. vote for candidate m (0 or 1)

DB_VOTES = []

### Okamoto-Uchiyama Cryptosystem

In [26]:
from random import randint

In [27]:
def gen_key(p: int, q: int) -> (int, int, int, int, int):
    '''Generate public and private keys used in
    okamoto-uchiyama cryptosystems from given
    prime numbers p and q.

    It's assumed that both p and q are prime
    numbers that have the same bit length.'''

    p_square = p ** 2

    n = (p_square) * q

    # Choose random integer g that satisfy
    # g ** (p-1) mod (p ** 2) != 1
    g = randint(2, n - 2)
    while pow(g, p - 1, p_square) == 1:
        g = randint(2, n - 2)

    h = pow(g, n, n)

    return n, g, h, p, q

In [28]:
def encrypt(vote: list[int], n: int, g: int, h: int) -> list[int]:
    '''Encrypt vote with public key (n, g, h)
    using okamoto-uchiyama cryptosystem.'''

    cipher_vote: list[int] = []

    for v in vote:
        r = randint(1, n - 1)
        c = (pow(g, v, n) * pow(h, r, n)) % n
        cipher_vote.append(c)

    return cipher_vote

In [29]:
def decrypt(cipher_vote: list[int], p: int, g: int) -> list[int]:
    '''Decrypt vote with key (p, g) using
    okamoto-uchiyama cryptosystem.'''

    p_square = p ** 2

    vote: list[int] = []
    
    for c in cipher_vote:
        a = (pow(c, p - 1, p_square) - 1) // p
        b = (pow(g, p - 1, p_square) - 1) // p

        v = (a * pow(b, -1, p)) % p

        vote.append(v)

    return vote

In [30]:
def add(vote_1: list[int], vote_2: list[int], n: int) -> list[int]:
    '''Perform addition between two cipher votes.

    It's assumed that both cipher votes have the
    same number of votes.'''

    res: list[int] = []

    for v1, v2 in zip(vote_1, vote_2):
        res.append((v1 * v2) % n)

    return res

## Testing

### Key Generation

In [31]:
# To make calculation easier, we'll use
# small prime numbers to generate key
# used in paillier cryptosystem

P = 5519
Q = 7013

In [32]:
n, g, h, p, q = gen_key(P, Q)
print(n, g, h, p, q)

213611498693 13401585519 189727378736 5519 7013


### Votes

Let's assume that there are two candidates to vote for. Therefore, each row in database for votes contain these information:

1. timestamps

2. voter's id

3. vote for candidate 1 (0 or 1)

4. vote for candidate 2 (0 or 1)


In [33]:
from hashlib import sha3_256
from datetime import datetime

In [34]:
VOTERS_ID: list[str] = [
    '1111111111111111', # voter 1
    '2222222222222222', # voter 2
    '3333333333333333', # voter 3
    '4444444444444444', # voter 4
    '5555555555555555'  # voter 5
]

VOTES_GIVEN: list = [
    [0, 1], # voter 1
    [1, 0], # voter 2
    [0, 1], # voter 3
    [1, 0], # voter 4
    [0, 1], # voter 5
]

for voter_id, vote in zip(VOTERS_ID, VOTES_GIVEN):
    ID: str = sha3_256(voter_id.encode('utf-8')).hexdigest().upper()
    vote_1, vote_2 = encrypt(vote, n, g, h)
    
    DB_VOTES.append((
        str(datetime.now()),
        ID,
        vote_1,
        vote_2
    ))

for i, vote in enumerate(DB_VOTES):
    print(f'vote 0{i+1}')
    print(vote)
    print()

vote 01
('2021-11-27 15:42:00.130523', '1AFD827639BD0919C0F788EB1C9F80AAABD13AC91949610CBDBBCA909401DD14', 49804642109, 103856844450)

vote 02
('2021-11-27 15:42:00.130523', 'C840939042E1596FE83C41E4B10DE197CEB490D4EF13E6F7A9621F25CB327435', 136985125956, 27195361484)

vote 03
('2021-11-27 15:42:00.130523', '057973DEF317E365BF64D675EA635AE5A16410902109E59BE7416022B2033129', 7352868113, 48346303935)

vote 04
('2021-11-27 15:42:00.130523', '4C68EF38B4D3D5697B85C3A696D1C4A1BE5AA6536756754371FA94C0AADFF411', 46338376586, 153206200880)

vote 05
('2021-11-27 15:42:00.131523', '380AF84015F83E74A2F6F63DD2AC97BF927822B4F90668FFC3200EF8740AA5B0', 201397904380, 61728595541)



In [35]:
sum_votes: list[int] = []

for i, vote in enumerate(DB_VOTES):
    vote_1, vote_2 = vote[2], vote[3]
    if i == 0:
        sum_votes.append(vote_1)
        sum_votes.append(vote_2)
    
    else:
        sum_votes = add(sum_votes, [vote_1, vote_2], n)

sum_votes

[109782504189, 6270883542]

In [36]:
decipher_vote = decrypt(sum_votes, p, g)
decipher_vote

[2, 3]